In [ ]:
#| default_exp handlers.maris_legacy_long

# MARIS Legacy

> This notebook contains a data pipeline (handler) that converts the master MARIS database dump into `NetCDF` format. It enables batch encoding of all legacy datasets into NetCDF.


Key functions of this handler:

- **Load data** from a MARIS dump file
- **Transform data** by applying various transformations to clean and normalize the data
- **Reshape data** from long to wide format
- **Encode data** into NetCDF files


The **result** is a set of NetCDF files, one for each unique reference ID in the input data.

## Packages import

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from tqdm import tqdm
from pathlib import Path
import fastcore.all as fc
import pandas as pd
import numpy as np

from marisco.callbacks import (
    Callback, 
    Transformer, 
    SanitizeLonLatCB, 
    EncodeTimeCB, 
    RenameColumnsCB, 
    SelectColumnsCB,
    UniqueIndexCB
)

from marisco.metadata import (
    GlobAttrsFeeder, 
    BboxCB, 
    DepthRangeCB,
    TimeRangeCB,
    ZoteroCB,KeyValuePairCB
    )

from marisco.configs import (
    NC_GROUPS,
    lut_path,
    cdl_cfg,
    cfg,
    nc_tpl_path,
    Enums, get_lut
    )

from marisco.encoders import NetCDFEncoder

In [ ]:
#| hide
import warnings
pd.set_option('display.max_rows', 100)
warnings.filterwarnings('ignore')

## Configuration and file paths

In [ ]:
# | exports
# fname_in = Path().home() / 'pro/data/maris/MARIS_exportSample_20240313.txt'
# dir_dest = '../../_data/output/dump'


fname_in = Path().home() / 'pro/data/maris/2024-11-08 MARIS_QA_shapetype_id=1.xlsx'
dir_dest = '../../_data/output/dump'

## Utils

In [ ]:
#| exports
class DataLoader:
    "Load specific MARIS dataset through its ref_id."
    LUT = {
        'Biota': 'BIOTA', 
        'Seawater': 'SEAWATER', 
        'Sediment': 'SEDIMENT', 
        'Suspended matter': 'SUSPENDED_MATTER'
    }

    def __init__(self, 
                 fname: str # Path to the MARIS global dump file
                 ):
        self.fname = fname
        self.df = None  # Lazy loading

    def _load_data(self):
        if self.df is None:
            self.df = pd.read_excel(self.fname)

    def __call__(self, 
                 ref_id: int # Reference ID of interest
                 ) -> dict: # Dictionary of dataframes
        self._load_data()
        filtered_df = self.df[self.df.ref_id == ref_id]
        return {self.LUT[name]: grp for name, grp in filtered_df.groupby('samptype') if name in self.LUT}


In [ ]:
#| exports
def get_zotero_key(dfs):
    "Retrieve Zotero key from MARIS dump."
    return dfs[next(iter(dfs))][['zoterourl']].iloc[0].values[0].split('/')[-1]

In [ ]:
#| exports
def get_fname(dfs):
    "Retrieve filename from MARIS dump."
    id, name = dfs[next(iter(dfs))][['ref_id', 'displaytext']].iloc[0]
    name = name.replace(',', '').replace('.', '').replace('-', ' ').split(' ')
    return '-'.join(([str(id)] + name)) + '.nc'

## Load data

In [ ]:
#|eval: false
dataloader = DataLoader(fname_in)
ref_id = 100 # Some other ref_id examples: OSPAR: 191, HELCOM: 100, 717 (only seawater)

dfs = dataloader(ref_id=ref_id)
print(f'keys: {dfs.keys()}')
dfs['SEDIMENT'].head()

keys: dict_keys(['BIOTA', 'SEAWATER', 'SEDIMENT'])


,sample_id,area_id,areaname,samptype_id,samptype,ref_id,displaytext,zoterourl,ref_note,datbase,...,profile_id,sampnote,ref_fulltext,ref_yearpub,ref_sampleTypes,LongLat,shiftedcoordinates,shiftedlong,shiftedlat,id
576000,398412,2374,Kattegat,3,Sediment,100,"HELCOM MORS, 2018",https://www.zotero.org/groups/2432820/maris/it...,"Assumed Cs137, originally reported as 138Cs.",HELCOM MORS 2018 Environmental database,...,NaN,DENMARK,"HELCOM MORS, 2018. Environmental database - He...",2018,"1,2,3","10.998,55.375",0xE6100000010C0000000000B04B401F85EB51B8FE2540,10.9975,55.375,576001
576001,398412,2374,Kattegat,3,Sediment,100,"HELCOM MORS, 2018",https://www.zotero.org/groups/2432820/maris/it...,"Assumed Cs137, originally reported as 138Cs.",HELCOM MORS 2018 Environmental database,...,NaN,DENMARK,"HELCOM MORS, 2018. Environmental database - He...",2018,"1,2,3","10.998,55.375",0xE6100000010C0000000000B04B401F85EB51B8FE2540,10.9975,55.375,576002
576002,398412,2374,Kattegat,3,Sediment,100,"HELCOM MORS, 2018",https://www.zotero.org/groups/2432820/maris/it...,"Assumed Cs137, originally reported as 138Cs.",HELCOM MORS 2018 Environmental database,...,NaN,DENMARK,"HELCOM MORS, 2018. Environmental database - He...",2018,"1,2,3","10.998,55.375",0xE6100000010C0000000000B04B401F85EB51B8FE2540,10.9975,55.375,576003
576003,398412,2374,Kattegat,3,Sediment,100,"HELCOM MORS, 2018",https://www.zotero.org/groups/2432820/maris/it...,"Assumed Cs137, originally reported as 138Cs.",HELCOM MORS 2018 Environmental database,...,NaN,DENMARK,"HELCOM MORS, 2018. Environmental database - He...",2018,"1,2,3","10.998,55.375",0xE6100000010C0000000000B04B401F85EB51B8FE2540,10.9975,55.375,576004
576004,398412,2374,Kattegat,3,Sediment,100,"HELCOM MORS, 2018",https://www.zotero.org/groups/2432820/maris/it...,"Assumed Cs137, originally reported as 138Cs.",HELCOM MORS 2018 Environmental database,...,NaN,DENMARK,"HELCOM MORS, 2018. Environmental database - He...",2018,"1,2,3","10.998,55.375",0xE6100000010C0000000000B04B401F85EB51B8FE2540,10.9975,55.375,576005


In [ ]:
ref_id = 724

## Data transformation pipeline

### Select columns

In [ ]:
#| exports
cois_renaming_rules = {
    'sample_id': 'SMP_ID',
    'latitude': 'LAT',
    'longitude': 'LON',
    'begperiod': 'TIME',
    'sampdepth': 'SMP_DEPTH',
    'totdepth': 'TOT_DEPTH',
    'uncertaint': 'UNC',
    'unit_id': 'UNIT',
    'detection': 'DL',
    'area_id': 'AREA',
    'species_id': 'SPECIES',
    'biogroup_id': 'BIO_GROUP',
    'bodypar_id': 'BODY_PART',
    'sedtype_id': 'SED_TYPE',
    'volume': 'VOL',
    'salinity': 'SAL',
    'temperatur': 'TEMP',
    'sampmet_id': 'SAMP_MET',
    'prepmet_id': 'PREP_MET',
    'counmet_id': 'COUNT_MET',
    'activity': 'VALUE',
    'nuclide_id': 'NUCLIDE',
    'sliceup': 'TOP',
    'slicedown': 'BOTTOM'
}

In [ ]:
#|eval: false
dfs = dataloader(ref_id=ref_id)
tfm = Transformer(dfs, cbs=[
    SelectColumnsCB(cois_renaming_rules)
    ])

print('Keys:', tfm().keys())
print('Columns:', tfm()['BIOTA'].columns)

Keys: dict_keys(['BIOTA'])
Columns: Index(['sample_id', 'latitude', 'longitude', 'begperiod', 'sampdepth',
       'totdepth', 'uncertaint', 'unit_id', 'detection', 'area_id',
       'species_id', 'biogroup_id', 'bodypar_id', 'sedtype_id', 'volume',
       'salinity', 'temperatur', 'sampmet_id', 'prepmet_id', 'counmet_id',
       'activity', 'nuclide_id', 'sliceup', 'slicedown'],
      dtype='object')


### Rename columns

In [ ]:
#|eval: false
dfs = dataloader(ref_id=ref_id)
tfm = Transformer(dfs, cbs=[
    SelectColumnsCB(cois_renaming_rules),
    RenameColumnsCB(cois_renaming_rules)
    ])

dfs_tfm = tfm()
print('Keys:', dfs_tfm.keys())
print('Columns:', dfs_tfm['BIOTA'].columns)

Keys: dict_keys(['BIOTA'])
Columns: Index(['SMP_ID', 'LAT', 'LON', 'TIME', 'SMP_DEPTH', 'TOT_DEPTH', 'UNC', 'UNIT',
       'DL', 'AREA', 'SPECIES', 'BIO_GROUP', 'BODY_PART', 'SED_TYPE', 'VOL',
       'SAL', 'TEMP', 'SAMP_MET', 'PREP_MET', 'COUNT_MET', 'VALUE', 'NUCLIDE',
       'TOP', 'BOTTOM'],
      dtype='object')


### Drop NaN only columns

In [ ]:
#| exports
class DropNAColumnsCB(Callback):
    "Drop variable containing only NaN or 'Not available' (id=0 in MARIS lookup tables)."
    def __init__(self, na_value=0): fc.store_attr()
    def isMarisNA(self, col): 
        return len(col.unique()) == 1 and col.iloc[0] == self.na_value
    
    def dropMarisNA(self, df):
        na_cols = [col for col in df.columns if self.isMarisNA(df[col])]
        return df.drop(labels=na_cols, axis=1)
        
    def __call__(self, tfm):
        for k in tfm.dfs.keys():
            tfm.dfs[k] = tfm.dfs[k].dropna(axis=1, how='all')
            tfm.dfs[k] = self.dropMarisNA(tfm.dfs[k])

In [ ]:
#|eval: false
dfs = dataloader(ref_id=ref_id)
tfm = Transformer(dfs, cbs=[
    SelectColumnsCB(cois_renaming_rules),
    RenameColumnsCB(cois_renaming_rules),
    DropNAColumnsCB()
    ])

dfs_tfm = tfm()
print('Keys:', dfs_tfm.keys())
print('Columns:', dfs_tfm['BIOTA'].columns)


Keys: dict_keys(['BIOTA'])
Columns: Index(['SMP_ID', 'LAT', 'LON', 'TIME', 'UNC', 'UNIT', 'DL', 'AREA', 'SPECIES',
       'BIO_GROUP', 'BODY_PART', 'COUNT_MET', 'VALUE', 'NUCLIDE'],
      dtype='object')


### Remap detection limit values

In [ ]:
#| exports
dl_name_to_id = lambda: get_lut(lut_path(), 'dbo_detectlimit.xlsx', key='name', value='id')

In [ ]:
#| eval: false
dl_name_to_id()

{'Not applicable': -1, 'Not Available': 0, '=': 1, '<': 2, 'ND': 3, 'DE': 4}

In [ ]:
#| exports
class SanitizeDetectionLimitCB(Callback):
    "Assign Detection Limit name to its id based on MARIS nomenclature."
    def __init__(self,
                 fn_lut=dl_name_to_id,
                 dl_name='DL'):
        fc.store_attr()

    def __call__(self, tfm):
        lut = self.fn_lut()
        for k in tfm.dfs.keys():
            tfm.dfs[k][self.dl_name] = tfm.dfs[k][self.dl_name].replace(lut)

In [ ]:
#|eval: false
dfs = dataloader(ref_id=ref_id)
tfm = Transformer(dfs, cbs=[
    SelectColumnsCB(cois_renaming_rules),
    RenameColumnsCB(cois_renaming_rules),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB()
    ])

dfs_tfm = tfm()
print('Keys:', dfs_tfm.keys())
print('Columns:', dfs_tfm['BIOTA'].columns)
print(f'{dfs_tfm["BIOTA"]["DL"].unique()}')
print(f'{dfs_tfm["BIOTA"].head()}')

Keys: dict_keys(['BIOTA'])
Columns: Index(['SMP_ID', 'LAT', 'LON', 'TIME', 'UNC', 'UNIT', 'DL', 'AREA', 'SPECIES',
       'BIO_GROUP', 'BODY_PART', 'COUNT_MET', 'VALUE', 'NUCLIDE'],
      dtype='object')
[1 2]
       SMP_ID       LAT         LON       TIME  UNC  UNIT  DL  AREA  SPECIES  \
20796  681755  36.35828  140.666057 1978-11-01  NaN     5   1  1908      445   
20797  681755  36.35828  140.666057 1978-11-01  NaN     5   1  1908      445   
20798  681821  36.35828  140.666057 1979-07-06  NaN     5   1  1908      445   
20799  681821  36.35828  140.666057 1979-07-06  NaN     5   1  1908      445   
20800  681821  36.35828  140.666057 1979-07-06  NaN     5   1  1908      445   

       BIO_GROUP  BODY_PART  COUNT_MET    VALUE  NUCLIDE  
20796          4          1         20    0.300       33  
20797          4          1         22    0.026       37  
20798          4          1          9  148.000      103  
20799          4          1         22    0.019       12  
20800         

### Parse and encode time

We remind that in `netCDF` format time need to be encoded as `integer` representing the number of seconds since a time of reference. In our case we chose `1970-01-01 00:00:00.0` as defined in `configs.ipynb`.



In [ ]:
#| exports
class ParseTimeCB(Callback):
    "Parse time column from MARIS dump."
    def __init__(self,
                 time_name='TIME'):
        fc.store_attr()
        
    def __call__(self, tfm):
        for k in tfm.dfs.keys():
            tfm.dfs[k][self.time_name] = pd.to_datetime(tfm.dfs[k][self.time_name], format='ISO8601')

In [ ]:
#|eval: false
dfs = dataloader(ref_id=ref_id)
tfm = Transformer(dfs, cbs=[
    SelectColumnsCB(cois_renaming_rules),
    RenameColumnsCB(cois_renaming_rules),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB(),
    ParseTimeCB(),
    EncodeTimeCB()
    ])

print(tfm()['BIOTA'])

        SMP_ID        LAT         LON        TIME    UNC  UNIT  DL  AREA  \
20796   681755  36.358280  140.666057   278726400    NaN     5   1  1908   
20797   681755  36.358280  140.666057   278726400    NaN     5   1  1908   
20798   681821  36.358280  140.666057   300067200    NaN     5   1  1908   
20799   681821  36.358280  140.666057   300067200    NaN     5   1  1908   
20800   681821  36.358280  140.666057   300067200    NaN     5   1  1908   
...        ...        ...         ...         ...    ...   ...  ..   ...   
944501  681221  36.188018  138.023379  1195084800  0.011     5   1  9999   
944502  681221  36.188018  138.023379  1195084800  0.021     5   1  9999   
944503  681222  36.188018  138.023379  1226880000  0.910     5   1  9999   
944504  681222  36.188018  138.023379  1226880000  0.010     5   1  9999   
944505  681222  36.188018  138.023379  1226880000  0.019     5   1  9999   

        SPECIES  BIO_GROUP  BODY_PART  COUNT_MET    VALUE  NUCLIDE  
20796       445   

### Sanitize coordinates

In [ ]:
#|eval: false
dfs = dataloader(ref_id=ref_id)
tfm = Transformer(dfs, cbs=[
    SelectColumnsCB(cois_renaming_rules),
    RenameColumnsCB(cois_renaming_rules),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB(),
    ParseTimeCB(),
    EncodeTimeCB(),
    SanitizeLonLatCB()
    ])

dfs_test = tfm()
dfs_test['BIOTA']

,SMP_ID,LAT,LON,TIME,UNC,UNIT,DL,AREA,SPECIES,BIO_GROUP,BODY_PART,COUNT_MET,VALUE,NUCLIDE
20796,681755,36.358280,140.666057,278726400,NaN,5,1,1908,445,4,1,20,0.300,33
20797,681755,36.358280,140.666057,278726400,NaN,5,1,1908,445,4,1,22,0.026,37
20798,681821,36.358280,140.666057,300067200,NaN,5,1,1908,445,4,1,9,148.000,103
20799,681821,36.358280,140.666057,300067200,NaN,5,1,1908,445,4,1,22,0.019,12
20800,681821,36.358280,140.666057,300067200,NaN,5,1,1908,445,4,1,20,0.240,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944501,681221,36.188018,138.023379,1195084800,0.011,5,1,9999,262,4,1,22,0.120,33
944502,681221,36.188018,138.023379,1195084800,0.021,5,1,9999,262,4,1,20,0.110,33
944503,681222,36.188018,138.023379,1226880000,0.910,5,1,9999,262,4,1,20,85.000,4
944504,681222,36.188018,138.023379,1226880000,0.010,5,1,9999,262,4,1,22,0.110,33


### Set unique index

In [ ]:
#|eval: false
dfs = dataloader(ref_id=ref_id)
tfm = Transformer(dfs, cbs=[
    SelectColumnsCB(cois_renaming_rules),
    RenameColumnsCB(cois_renaming_rules),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB(),
    ParseTimeCB(),
    EncodeTimeCB(),
    SanitizeLonLatCB(),
    UniqueIndexCB()
    ])

dfs_test = tfm()
dfs_test['BIOTA']

,ID,SMP_ID,LAT,LON,TIME,UNC,UNIT,DL,AREA,SPECIES,BIO_GROUP,BODY_PART,COUNT_MET,VALUE,NUCLIDE
0,0,681755,36.358280,140.666057,278726400,NaN,5,1,1908,445,4,1,20,0.300,33
1,1,681755,36.358280,140.666057,278726400,NaN,5,1,1908,445,4,1,22,0.026,37
2,2,681821,36.358280,140.666057,300067200,NaN,5,1,1908,445,4,1,9,148.000,103
3,3,681821,36.358280,140.666057,300067200,NaN,5,1,1908,445,4,1,22,0.019,12
4,4,681821,36.358280,140.666057,300067200,NaN,5,1,1908,445,4,1,20,0.240,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86646,86646,681221,36.188018,138.023379,1195084800,0.011,5,1,9999,262,4,1,22,0.120,33
86647,86647,681221,36.188018,138.023379,1195084800,0.021,5,1,9999,262,4,1,20,0.110,33
86648,86648,681222,36.188018,138.023379,1226880000,0.910,5,1,9999,262,4,1,20,85.000,4
86649,86649,681222,36.188018,138.023379,1226880000,0.010,5,1,9999,262,4,1,22,0.110,33


## Encode to NetCDF

In [ ]:
#|eval: false
dfs = dataloader(ref_id=ref_id)
tfm = Transformer(dfs, cbs=[
    SelectColumnsCB(cois_renaming_rules),
    RenameColumnsCB(cois_renaming_rules),
    DropNAColumnsCB(),
    SanitizeDetectionLimitCB(),
    ParseTimeCB(),
    EncodeTimeCB(),
    SanitizeLonLatCB(),
    UniqueIndexCB()
    ])

dfs_tfm = tfm()
tfm.logs

['Select columns of interest.',
 'Renaming variables to MARIS standard names.',
 "Drop variable containing only NaN or 'Not available' (id=0 in MARIS lookup tables).",
 'Assign Detection Limit name to its id based on MARIS nomenclature.',
 'Parse time column from MARIS dump.',
 'Encode time as seconds since epoch.',
 'Drop rows with invalid longitude & latitude values. Convert `,` separator to `.` separator.',
 'Set unique index for each group.']

In [ ]:
#| exports
kw = ['oceanography', 'Earth Science > Oceans > Ocean Chemistry> Radionuclides',
      'Earth Science > Human Dimensions > Environmental Impacts > Nuclear Radiation Exposure',
      'Earth Science > Oceans > Ocean Chemistry > Ocean Tracers, Earth Science > Oceans > Marine Sediments',
      'Earth Science > Oceans > Ocean Chemistry, Earth Science > Oceans > Sea Ice > Isotopes',
      'Earth Science > Oceans > Water Quality > Ocean Contaminants',
      'Earth Science > Biological Classification > Animals/Vertebrates > Fish',
      'Earth Science > Biosphere > Ecosystems > Marine Ecosystems',
      'Earth Science > Biological Classification > Animals/Invertebrates > Mollusks',
      'Earth Science > Biological Classification > Animals/Invertebrates > Arthropods > Crustaceans',
      'Earth Science > Biological Classification > Plants > Macroalgae (Seaweeds)']

In [ ]:
#| exports
def get_attrs(tfm, zotero_key, kw=kw):
    "Retrieve global attributes from MARIS dump."
    return GlobAttrsFeeder(tfm.dfs, cbs=[
        BboxCB(),
        DepthRangeCB(),
        TimeRangeCB(),
        ZoteroCB(zotero_key, cfg=cfg()),
        KeyValuePairCB('keywords', ', '.join(kw)),
        KeyValuePairCB('publisher_postprocess_logs', ', '.join(tfm.logs))
        ])()

In [ ]:
#|eval: false
get_attrs(tfm, zotero_key='3W354SQG', kw=kw)

{'geospatial_lat_min': '30.4358333333333',
 'geospatial_lat_max': '65.75',
 'geospatial_lon_min': '9.63333333333333',
 'geospatial_lon_max': '53.5',
 'geospatial_bounds': 'POLYGON ((9.63333333333333 53.5, 30.4358333333333 53.5, 30.4358333333333 65.75, 9.63333333333333 65.75, 9.63333333333333 53.5))',
 'geospatial_vertical_max': '437.0',
 'geospatial_vertical_min': '-1.0',
 'time_coverage_start': '1984-01-10T00:00:00',
 'time_coverage_end': '2018-12-14T00:00:00',
 'title': 'Radioactivity Monitoring of the Irish Marine Environment 1991 and 1992',
 'summary': '',
 'creator_name': '[{"creatorType": "author", "firstName": "A.", "lastName": "McGarry"}, {"creatorType": "author", "firstName": "S.", "lastName": "Lyons"}, {"creatorType": "author", "firstName": "C.", "lastName": "McEnri"}, {"creatorType": "author", "firstName": "T.", "lastName": "Ryan"}, {"creatorType": "author", "firstName": "M.", "lastName": "O\'Colmain"}, {"creatorType": "author", "firstName": "J.D.", "lastName": "Cunningham"}

In [ ]:
#| exports
def encode(fname_in, fname_out, dataloader=None, **kwargs):
    if dataloader is None: dataloader = DataLoader(fname_in)._load_data()
    ref_ids = kwargs.get('ref_ids', dataloader.df.ref_id.unique())
    print('Encoding ...')
    for ref_id in tqdm(ref_ids, leave=False):
        dfs = dataloader(ref_id=ref_id)
        print(get_fname(dfs))
        tfm = Transformer(dfs, cbs=[
            SelectColumnsCB(cois_renaming_rules),
            RenameColumnsCB(cois_renaming_rules),
            DropNAColumnsCB(),
            SanitizeDetectionLimitCB(),
            ParseTimeCB(),
            EncodeTimeCB(),
            SanitizeLonLatCB(),
            UniqueIndexCB()
            ])
        
        tfm()
        encoder = NetCDFEncoder(tfm.dfs, 
                                dest_fname=Path(fname_out) / get_fname(dfs), 
                                global_attrs=get_attrs(tfm, zotero_key=get_zotero_key(dfs), kw=kw),
                                verbose=kwargs.get('verbose', False)
                                )
        encoder.encode()

### Single dataset

In [ ]:
#|eval: false
ref_id = 727
encode(
    fname_in,
    dir_dest,
    dataloader=dataloader,
    verbose=True, 
    ref_ids=[ref_id])

Encoding ...


  0%|          | 0/1 [00:00<?, ?it/s]

727-Muir-et-al-2017.nc


--------------------------------------------------------------------------------
Group: biota, Variable: id
--------------------------------------------------------------------------------
Group: biota, Variable: lon
--------------------------------------------------------------------------------
Group: biota, Variable: lat
--------------------------------------------------------------------------------
Group: biota, Variable: time
--------------------------------------------------------------------------------
Group: biota, Variable: area
--------------------------------------------------------------------------------
Group: biota, Variable: smp_id
--------------------------------------------------------------------------------
Group: biota, Variable: nuclide
--------------------------------------------------------------------------------
Group: biota, Variable: value
--------------------------------------------------------------------------------
Group: biota, Variable: unit
--------

ValueError: trying to assign illegal value to Enum variable

### All datasets

In [ ]:
#|eval: false
encode(
    fname_in, 
    dir_dest, 
    dataloader=dataloader,
    verbose=False)

Encoding ...


  0%|          | 0/475 [00:00<?, ?it/s]

402-CCHDO-2018.nc


  0%|          | 1/475 [00:01<15:22,  1.95s/it]

374-Ostlund-et-al-1987.nc


...

 44%|████▍     | 208/475 [03:29<03:36,  1.23it/s]

173-Hurtado-Bermudez-et-al-2017.nc


 44%|████▍     | 209/475 [03:30<03:36,  1.23it/s]

727-Muir-et-al-2017.nc


ValueError: trying to assign illegal value to Enum variable